In [5]:
import pandas as pd
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [6]:
df=pd.read_csv("bike.csv")
df.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [7]:
df_sub=df.loc[:,"season":"casual"]
df_sub.head(2)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual
0,1,0,0,1,9.84,14.395,81,0.0,3
1,1,0,0,1,9.02,13.635,80,0.0,8


In [8]:
"causal~season+holiday+workingday+...+windspeed" #종속변수 ~ 독립변수는 +로 이어짐

'causal~season+holiday+workingday+...+windspeed'

In [9]:
"casual~" + "season+holiday"

'casual~season+holiday'

In [12]:
"casual~"+"+".join(df_sub.columns)

'casual~season+holiday+workingday+weather+temp+atemp+humidity+windspeed+casual'

In [13]:
"casual~"+"+".join(df_sub.columns[:-1]) #변수명 적기 쉽게 하기

'casual~season+holiday+workingday+weather+temp+atemp+humidity+windspeed'

In [17]:
formula="casual~"+"+".join(df_sub.columns[:-1])
y,X=dmatrices(formula,data=df_sub,return_type="dataframe")

In [18]:
y.head(2)

,casual
0,3.0
1,8.0


In [19]:
X.head(2)

,Intercept,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,1.0,1.0,0.0,0.0,1.0,9.84,14.395,81.0,0.0
1,1.0,1.0,0.0,0.0,1.0,9.02,13.635,80.0,0.0


In [23]:
df_vif=pd.DataFrame()
df_vif["colname"]=X.columns
df_vif["VIF"]=[vif(X.values,i)for i in range(X.shape[1])]
df_vif

,colname,VIF
0,Intercept,34.029472
1,season,1.137211
2,holiday,1.069731
3,workingday,1.071196
4,weather,1.236150
5,temp,35.516012
6,atemp,35.550831
7,humidity,1.425034
8,windspeed,1.195704


In [24]:
df_sub=pd.concat([df.loc[:,"season":"temp"], #atemp 변수 빼고 만드는 트릭
                 df.loc[:,"humidity":"casual"]],
                axis=1)
df_sub.head(2)

,season,holiday,workingday,weather,temp,humidity,windspeed,casual
0,1,0,0,1,9.84,81,0.0,3
1,1,0,0,1,9.02,80,0.0,8


In [25]:
formula="casual~"+"+".join(df_sub.columns[:-1]) #변수명 너무 많아서 못쓸때 활용하면 굿
y,X=dmatrices(formula,data=df_sub,return_type="dataframe")

df_vif=pd.DataFrame()
df_vif["colname"]=X.columns
df_vif["VIF"]=[vif(X.values,i)for i in range(X.shape[1])]
df_vif

,colname,VIF
0,Intercept,31.375118
1,season,1.136866
2,holiday,1.068094
3,workingday,1.070025
4,weather,1.235251
5,temp,1.089028
6,humidity,1.421256
7,windspeed,1.149650


In [27]:
df.corr().round(2) #temp 와 atemp 의 상관계수가 매우 높다 다중공선성

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
season,1.00,0.03,-0.01,0.01,0.26,0.26,0.19,-0.15,0.10,0.16,0.16
holiday,0.03,1.00,-0.25,-0.01,0.00,-0.01,0.00,0.01,0.04,-0.02,-0.01
workingday,-0.01,-0.25,1.00,0.03,0.03,0.02,-0.01,0.01,-0.32,0.12,0.01
weather,0.01,-0.01,0.03,1.00,-0.06,-0.06,0.41,0.01,-0.14,-0.11,-0.13
temp,0.26,0.00,0.03,-0.06,1.00,0.98,-0.06,-0.02,0.47,0.32,0.39
atemp,0.26,-0.01,0.02,-0.06,0.98,1.00,-0.04,-0.06,0.46,0.31,0.39
humidity,0.19,0.00,-0.01,0.41,-0.06,-0.04,1.00,-0.32,-0.35,-0.27,-0.32
windspeed,-0.15,0.01,0.01,0.01,-0.02,-0.06,-0.32,1.00,0.09,0.09,0.10
casual,0.10,0.04,-0.32,-0.14,0.47,0.46,-0.35,0.09,1.00,0.50,0.69
registered,0.16,-0.02,0.12,-0.11,0.32,0.31,-0.27,0.09,0.50,1.00,0.97


In [28]:
df_dum=pd.get_dummies(df,columns=["season"],drop_first =True)
df_dum.head(2)

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,season_2,season_3,season_4
0,2011-01-01 00:00:00,0,0,1,9.84,14.395,81,0.0,3,13,16,0,0,0
1,2011-01-01 01:00:00,0,0,1,9.02,13.635,80,0.0,8,32,40,0,0,0


1번 Price를 종속변수로 하고 나머지 수치형 변수를 독립변수로 했을 때 다중 공선성의 문제가 있다고 판단되는 변수의 개수?<br>
diamonds.csv파일 사용

In [29]:
df=pd.read_csv("diamonds.csv")
df.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


In [30]:
df_sub1=pd.concat([df.loc[:,"carat"], #atemp 변수 빼고 만드는 트릭
                 df.loc[:,"depth":"z"]],
                axis=1)
df_sub1.head(2)

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31


In [32]:
df_sub2 = df.iloc[:,[6,0,4,5,7,8,9]] # 이방식으로 해도 위와 같이 나옴
df_sub2.head(2) #price 가장 앞에 배치하여 formula 하기 쉽게 만들기

,price,carat,depth,table,x,y,z
0,326,0.23,61.5,55.0,3.95,3.98,2.43
1,326,0.21,59.8,61.0,3.89,3.84,2.31


In [37]:
y,X=dmatrices("price~"+"+".join(df_sub2.columns[1:]),data=df_sub2,return_type="dataframe")

df_vif=pd.DataFrame()
df_vif["vars"]=X.columns
df_vif["VIF"]=[vif(X.values,i)for i in range(X.shape[1])]
df_vif

,vars,VIF
0,Intercept,4821.696350
1,carat,21.602712
2,depth,1.496590
3,table,1.143225
4,x,56.187704
5,y,20.454295
6,z,23.530049


2번 price를 종속변수로 하고 carat과 depth를 독립변수로 하여 생성한 선형회귀모델을 사용하여<br>
알아본 carat이 1이고 depth가 60, table이 55인 다이아몬드의 가격은 얼마인가?

In [39]:
df=pd.read_csv("diamonds.csv")
df.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


In [40]:
from statsmodels.formula.api import ols

In [41]:
model=ols(formula="price~carat+depth",data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.851
Model:                            OLS   Adj. R-squared:                  0.851
Method:                 Least Squares   F-statistic:                 1.536e+05
Date:                Thu, 25 Nov 2021   Prob (F-statistic):               0.00
Time:                        12:46:02   Log-Likelihood:            -4.7249e+05
No. Observations:               53940   AIC:                         9.450e+05
Df Residuals:                   53937   BIC:                         9.450e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   4045.3332    286.205     14.134      0.000    3484.368    4606.298
carat       7765.1407     14.009    554.282      0.000    7737.682    7792.599
depth       -102.1653      4.635    -22.041      0.000    -111.251     -93.080
==============================================================================
Omnibus:                    14148.858   Durbin-Watson:                   0.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           148236.675
Skew:                           0.962   Prob(JB):                         0.00
Kurtosis:                      10.890   Cond. No.                     2.66e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.66e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [42]:
df_test=pd.DataFrame({"carat":[1],
                     "depth":[60],  #carat 이 1 depth가 60 table이 55인 dataframe을 직접 만든다 
                     "table":[55]})
df_test

,carat,depth,table
0,1,60,55


In [43]:
model.predict(df_test)

0    5680.554517
dtype: float64

3번 price를 종속변수로 하고 carat,color,depth를 독립변수로 하여 생성한 선형 회귀 모델을 사용하여<br>
알아본 carat이 1이고 depth가 50, color가 E인 다이아몬드의 가격은 얼마인가?

답지

In [66]:
df=pd.read_csv("diamonds.csv")
df.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


In [67]:
df_sub=df.loc[:,["price","carat","color","depth"]]

In [68]:
df_dum=pd.get_dummies(df_sub,columns=["color"],drop_first=True)
df_dum.head(2)

,price,carat,depth,color_E,color_F,color_G,color_H,color_I,color_J
0,326,0.23,61.5,1,0,0,0,0,0
1,326,0.21,59.8,1,0,0,0,0,0


In [69]:
model=ols(formula="price~"+"+".join(df_dum.columns[:1]),data=df_dum).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 6.124e+33
Date:                Thu, 25 Nov 2021   Prob (F-statistic):               0.00
Time:                        13:15:35   Log-Likelihood:             1.2806e+06
No. Observations:               53940   AIC:                        -2.561e+06
Df Residuals:                   53938   BIC:                        -2.561e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -8.47e-12   7.16e-14   -118.311      0.000   -8.61e-12   -8.33e-12
price          1.0000   1.28e-17   7.83e+16      0.000       1.000       1.000
==============================================================================
Omnibus:                    15542.219   Durbin-Watson:                   0.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            36733.156
Skew:                          -1.638   Prob(JB):                         0.00
Kurtosis:                       5.369   Cond. No.                     7.87e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.87e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [70]:
df_test=df_dum.iloc[[0],]
df_test

,price,carat,depth,color_E,color_F,color_G,color_H,color_I,color_J
0,326,0.23,61.5,1,0,0,0,0,0


In [71]:
df_test["carat"]=1
df_test["depth"]=50
df_test

<ipython-input-71-09f300b853dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["carat"]=1
<ipython-input-71-09f300b853dd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["depth"]=50


,price,carat,depth,color_E,color_F,color_G,color_H,color_I,color_J
0,326,1,50,1,0,0,0,0,0


In [72]:
model.predict(df_test)

0    326.0
dtype: float64

나의 답

In [62]:
#model=ols(formula="price~carat+color+depth",data=df).fit()
#model.summary()

In [59]:
#df_test=pd.DataFrame({"carat":[1],
#                    "depth":[50], 
#                     "color":["E"]})
#df_test

In [60]:
#model.predict(df_test)